In [1]:
## Parameters specific to where your folders are and your data
parameter_file = 'params/params.yaml'
import yaml
import sys

with open(parameter_file,'r') as f:
    doc = yaml.load(f)

#p = dic2obj(**doc)

data_folder = doc['data_folder']
tissues = doc['tissues'].split(',')
sys.path.append(doc['pipeline_path'])
ref_fa = doc['ref_fa']
anno_gff=doc['annotation']


tss_annotation = doc['tss_annotation']


import os
os.chdir
import sys
import pandas as pd
import matplotlib
import seaborn as sns
import pickle
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from itertools import product
import glob
import re
from matplotlib_venn import venn2
from matplotlib import rcParams
import inspect
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sys.setrecursionlimit(3000)
%load_ext autoreload
%autoreload 2
rcParams['figure.figsize'] = 8, 6
import tqdm

from os.path import basename
##mpl.use('Agg')
#mpl.style.use('ggplot')
#mpl.style.use('fivethirtyeight')
from Homer import *

print('Number of tissues: ',len(tissues))

('Number of tissues: ', 15)


In [2]:
import annotation

## Create histograms of coverage centered on peaks in RefSeq and Experimental peaks

In [3]:
import time

In [ ]:
mRNA_peak_file = '/data/isshamie/genome/picr_final/mRNA_final.peak'
out_dir = 'Results/coverage_histograms'

if not os.path.exists(out_dir):
    os.mkdir(out_dir)


for t in tissues:
    for s in ['GROCap','START']:
        curr_time = time.time()
        
        # Get the tag and peak files
        tag_files = 'f03_tags/*%s*%s*/' % (t,s)
        tag_files = glob.glob(tag_files) 
        peak_files = 'f04_peaks/*%s*%s*' % (t,s)
        peak_files = glob.glob(peak_files)
        if len(tag_files) != 0:
            curr_dir = os.path.join(out_dir,s + '_' + t)
            if not os.path.exists(curr_dir):
                os.mkdir(curr_dir)
            for i in peak_files:
                ## Create a peak file of refseq relative to peak file
                mrna_filt = os.path.join(curr_dir,os.path.basename(i)+'_mRNA')
                print('mrna_filt',mrna_filt)
                print('input to mrna_peak',i)
                if not os.path.exists(mrna_filt):
                    peakFileToPeakFile(mRNA_peak_file,i,distance=1000,f_save=mrna_filt)
                for j in tag_files:
                    # Refseq 
                    output_file_mrna = os.path.join(curr_dir,os.path.basename(i) + '_' + os.path.basename(j) + '.hist.mrna')
                    print('output histogram mrna',output_file_mrna)
                    hist(j,output_file_mrna,ref_fa,anno_gff,mode='peak',peak=mrna_filt+'filt',region=4000,res=25,pc=3)
                    hist_plot(output_file_mrna)
                    heat_plot(output_file_mrna+'MatS',save_f = output_file_mrna + '_heat.png' )
                    
                    # Experimental 
                    output_file_exp = os.path.join(curr_dir,os.path.basename(i) + '_' + os.path.basename(j) + '.hist.exp')
                    print('output histogram exp',output_file_exp)
                    hist(j,output_file_exp,ref_fa,anno_gff,mode='peak',peak=i,region=4000,res=25,pc=3)
                    hist_plot(output_file_exp)
                    heat_plot(output_file_exp+'MatS',save_f = output_file_exp + '_heat.png' )                    
        print('Time elapsed', time.time()-curr_time)

('Time elapsed', 0.0005578994750976562)
('Time elapsed', 0.000614166259765625)
('mrna_filt', 'Results/coverage_histograms/GROCap_BMDMwt/BMDMwt_GROCap1_and_BMDMwt_GRO1_mRNA')
('input to mrna_peak', 'f04_peaks/BMDMwt_GROCap1_and_BMDMwt_GRO1')
('output histogram mrna', 'Results/coverage_histograms/GROCap_BMDMwt/BMDMwt_GROCap1_and_BMDMwt_GRO1_.hist.mrna')
annotatePeaks.pl Results/coverage_histograms/GROCap_BMDMwt/BMDMwt_GROCap1_and_BMDMwt_GRO1_mRNAfilt /data/isshamie/genome/picr_final/picr.fa -gff /data/isshamie/genome/picr_final/final.gff3 -fragLength 1 -size 4000 -hist 25 -d f03_tags/BMDMwt_GROCap1/ -pc 3 > Results/coverage_histograms/GROCap_BMDMwt/BMDMwt_GROCap1_and_BMDMwt_GRO1_.hist.mrna
annotatePeaks.pl Results/coverage_histograms/GROCap_BMDMwt/BMDMwt_GROCap1_and_BMDMwt_GRO1_mRNAfilt /data/isshamie/genome/picr_final/picr.fa -gff /data/isshamie/genome/picr_final/final.gff3 -fragLength 1 -size 4000 -hist 25 -histNorm 100 -d f03_tags/BMDMwt_GROCap1/ -pc 3 > Results/coverage_histograms/GR

In [ ]:
mRNA_peak_file = '/data/isshamie/genome/picr_final/mRNA_final.peak'
out_dir = 'Results/coverage_histograms'

if not os.path.exists(out_dir):
    os.mkdir(out_dir)


for t in tissues:
    print(t)
    for s in ['GROCap','START']:
        curr_time = time.time()
        
        # Get the tag and peak files
        tag_files = 'f03_tags/*%s*%s*/' % (t,s)
        tag_files = glob.glob(tag_files) 
        peak_files = 'Results/tss_annotation_peaks/*%s*%s*' % (t,s)
        peak_files = glob.glob(peak_files)
        if len(tag_files) != 0:
            curr_dir = os.path.join(out_dir,s + '_' + t)
            if not os.path.exists(curr_dir):
                os.mkdir(curr_dir)
            for i in peak_files:
                ## Create a peak file of refseq relative to peak file
                mrna_filt = os.path.join(curr_dir,os.path.basename(i.split('.',1)[0])+'_mRNA')
                print('mrna_filt',mrna_filt)
                print('input to mrna_peak',i)
                if not os.path.exists(mrna_filt + 'filt'):
                    peakFileToPeakFile(mRNA_peak_file,i,distance=1000,f_save=mrna_filt)
                for j in tag_files:
                    # Refseq 
                    output_file_mrna = os.path.join(curr_dir,os.path.basename(i.split('.',1)[0]) + '.hist.mrna')
                    print('output histogram mrna',output_file_mrna)
                    hist(j,output_file_mrna,ref_fa,anno_gff,mode='peak',peak=mrna_filt+'filt',region=4000,res=25,pc=3)
                    hist_plot(output_file_mrna)
                    heat_plot(output_file_mrna+'MatS',save_f = output_file_mrna + '.heat.png' )
                    
                    # Experimental 
                    output_file_exp = os.path.join(curr_dir,os.path.basename(i.split('.',1)[0]) + '.hist.exp')
                    print('output histogram exp',output_file_exp)
                    hist(j,output_file_exp,ref_fa,anno_gff,mode='peak',peak=i,region=4000,res=25,pc=3)
                    hist_plot(output_file_exp)
                    heat_plot(output_file_exp+'MatS',save_f = output_file_exp + '.heat.png' )                    
        print('Time elapsed', time.time()-curr_time)

BMDM1hKLA
('Time elapsed', 0.00090789794921875)
('Time elapsed', 0.0005781650543212891)
BMDMwt
('mrna_filt', 'Results/coverage_histograms/GROCap_BMDMwt/sample_BMDMwt_GROCap1_and_BMDMwt_GRO1_mRNA')
('input to mrna_peak', 'Results/tss_annotation_peaks/sample_BMDMwt_GROCap1_and_BMDMwt_GRO1.tsv')
('output histogram mrna', 'Results/coverage_histograms/GROCap_BMDMwt/sample_BMDMwt_GROCap1_and_BMDMwt_GRO1.hist.mrna')
annotatePeaks.pl Results/coverage_histograms/GROCap_BMDMwt/sample_BMDMwt_GROCap1_and_BMDMwt_GRO1_mRNAfilt /data/isshamie/genome/picr_final/picr.fa -gff /data/isshamie/genome/picr_final/final.gff3 -fragLength 1 -size 4000 -hist 25 -d f03_tags/BMDMwt_GROCap1/ -pc 3 > Results/coverage_histograms/GROCap_BMDMwt/sample_BMDMwt_GROCap1_and_BMDMwt_GRO1.hist.mrna
annotatePeaks.pl Results/coverage_histograms/GROCap_BMDMwt/sample_BMDMwt_GROCap1_and_BMDMwt_GRO1_mRNAfilt /data/isshamie/genome/picr_final/picr.fa -gff /data/isshamie/genome/picr_final/final.gff3 -fragLength 1 -size 4000 -hist 25 -